# Data Collecting Project
## Studi Kasus: Twitter Data Crawling

In [ ]:
# install library Tweepy

import sys
!{sys.executable} -m pip install tweepy

In [ ]:
import tweepy

# setting key and token
consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""

In [ ]:
# membuat object authentication ke API Twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# mensetting access token and secret key
auth.set_access_token(access_token, access_token_secret)
# melakukan authentication ke Twitter API
api = tweepy.API(auth) 

In [ ]:
# melakukan crawling ke semua data public di home timeline
public_tweets = api.home_timeline()

for tweets in public_tweets:
    print(tweets.text)

In [ ]:
#melihat struktur JSON sebuah data tweet
public_tweets[0]._json

In [ ]:
# melakukan crawling 10 data public di home timeline 
for tweet in tweepy.Cursor(api.home_timeline).items(10):
    print(tweet.text)

In [ ]:
# menampilkan nama-nama friends
for friend in tweepy.Cursor(api.friends).items():
    print(friend.name)

In [ ]:
# mengcrawl data tweets kita sendiri
for tweet in tweepy.Cursor(api.user_timeline).items():
    print(tweet.text)

In [ ]:
# mengcrawl 5 data tweets dari 'nytimes'
for tweet in api.user_timeline(id="nytimes", count=5):
   print(tweet.text)

In [ ]:
import json

# menampilkan 5 data hasil crawling detikcom ke dalam format JSON
for tweet in api.user_timeline(id="detikcom", count=1):
    print(json.dumps(tweet._json))

In [ ]:
# mengambil tweets dengan taggar tertentu dg search

for tweet in tweepy.Cursor(api.search,q="#bigdata",
                           lang="en",
                           since="2019-01-01").items(1):
    print(tweet.text)

In [ ]:
# mengambil item tertentu dari JSON tweets ke list
tweets_data = []
for tweet in tweepy.Cursor(api.search,q="#bigdata",
                           lang="en",
                           since="2019-01-01").items(5):
    tweets_data.append([tweet.author.screen_name,
                        tweet.lang,
                        tweet.created_at,
                        tweet.favorite_count,
                        tweet.retweet_count,
                        tweet.text,
                        tweet.user.location])
print(tweets_data)

In [ ]:
# convert list to Dataframe
import pandas as pd
tweets_pd = pd.DataFrame(tweets_data,
                         columns=['screen_name', 'lang', 'created_at', 'fav_count', 'retweet_count', 'text'])
display(tweets_pd)

In [ ]:
# mengkonversi dataframe ke file csv
bytes_to_write = tweets_pd.to_csv(index = None, header=True).encode()

# letakkan file csv ke bucket S3
import s3fs
s3 = s3fs.S3FileSystem()

with s3.open('rosihanari/data_twitter.csv', 'wb') as f:
    f.write(bytes_to_write)

In [ ]:
# mengambil data streaming dari twitter dari keyword tertentu
# data stream dibaca dalam format JSON dan langsung disimpan di Amazon S3

import tweepy
import json
from tweepy.streaming import StreamListener
import s3fs
s3 = s3fs.S3FileSystem()

json_temp = []

class MyStreamListener(tweepy.StreamListener):
    
    def __init__(self, api=None):
        super(MyStreamListener, self).__init__()
        self.num_tweets = 0

    def on_status(self, tweet):
        
        # membatasi jumlah data streaming tweets (n = 5)
        self.num_tweets += 1
        if self.num_tweets < 5:
            # mencetak text status hasil streaming
            print(tweet.text)
            json_temp.append(tweet._json)
            # menyimpan setiap JSON hasil streaming ke S3
            with s3.open('rosihanari/data_twitter_json.json', 'ab') as f:
                f.write(json.dumps(json_temp).encode('utf-8'))
            return True
        else:
            return False

myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

# streaming keyword='python'
myStream.filter(track=['python'])

## Project

Lakukan data collecting terhadap Twitter melalui streaming dengan ketentuan sbb:
- Streaming dilakukan pada taggar **#BigData**. 
- Jumlah data yang dicollect sebanyak 50 buah. 
- Data yang dicollect adalah '**screen_name**', '**lang**', '**created_at**', '**retweet_count**', '**text**', '**location**'
- Tampung data hasil streaming ke dalam list
- Konversikan data list hasil streaming ke dalam dataframe
- Simpan dataframe hasil streaming ke dalam Amazon S3 dalam format CSV
- Lakukan streaming (50 buah data) pada data Twitter dg keyword **#Senin**, data streaming disimpan di file **SeninTwitter.json** (Data stream yang disimpan adalah JSON utuh)